<a href="https://colab.research.google.com/github/syunar/superai-ss3-level1-hackhathon/blob/main/superai_ss3_hack2_iot_temp_prediction_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

-	Install dependencies
    -	Numpy pandas -> to work with dataframe
    -	Seaborn, matplotlib -> to plot graph
    -	tensorflow.keras sklearn -> to work with DL and ML
-	Setting for kaggle api
-	Download dataset from kaggle
-	EDA
    -	Check Data type
    -	Check missing values
    -	Check duplicated values
    -	Check outliers
    -	Check correlation
-	Data Preprocessing
    -	Drop missing values on temp column (have 1)
    -	Drop column pm10 and pm2.5 which have missing values > 15% of records
    -	Drop outliers on wind_speed column filter only < 15000
    -	Feature Engineering
        - hour: เวลาในแต่ละวัน เช่น 8โมง 9โมง บ่ายโมง บ่ายสอง
        - month: เดือนที่เท่าไหร่
        - day: วันที่เท่าไหร่ใน 1 เดือน ตอนหลังมาคิดอีกทีควรใช้ วันในรอบปี น่าจะได้ผลดีขึ้น
        - one hot encoding: mac, station_name, tambon_namt, amphur_namt, province_namt, month, hour, day
        - drop คอลัมน์ที่ไม่ได้ใช้ หรือคอลัมน์ที่แปลงเป็น one hot แล้ว หรือคอลัมน์ที่ มีทิศทางเดียวกัน เช่น station id กับ station name
-	Build function called ‘prepData’ input raw_dataframe output is clean_dataframe
-	Clean train dataset and test dataset from kaggle
-	Split Train Dataset
-	Create model by using basic deep neural network (DNN)
    -	Normalization
    -	Use dropout ก่อนสุดท้ายเพื่อลด overfitting
    -	MAE as loss function
    -	Learning rate 0.0002
    -	Stoploss patience = 3
-	Train model
-	Test model on test data which split from train dataset
    - Acc 0.75
-	Predict on test dataset
-	Submission to kaggle


# Install Dependecies

In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/superai_hackhaton1_image/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c ss3-hackathon-online-iots-and-robotics
!unzip ss3-hackathon-online-iots-and-robotics.zip

  0% 0.00/263k [00:00<?, ?B/s]
100% 263k/263k [00:00<00:00, 67.3MB/s]
Archive:  ss3-hackathon-online-iots-and-robotics.zip
  inflating: IOT_Submit.csv          
  inflating: IOT_Test.csv            
  inflating: IOT_Train.csv           


In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
pd.set_option('float_format', '{:,.2f}'.format)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as ticker # change date format on chart

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


In [ ]:
!wget -q https://github.com/Phonbopit/sarabun-webfont/raw/master/fonts/thsarabunnew-webfont.ttf
import matplotlib as mpl
mpl.font_manager.fontManager.addfont('thsarabunnew-webfont.ttf')
mpl.rc('font', family='TH Sarabun New', size=20)
#plt.style.use('ggplot') # setting default style
#plt.rcParams.update({'font.size': 15})

In [ ]:
df = pd.read_csv('/content/IOT_Train.csv')

# EDA

## Basic Stats

In [ ]:
print(df.describe().T)

                  count       mean        std        min        25%        50%        75%        max
tambon_code   13,635.00 452,665.90 213,925.10 102,601.00 300,903.00 380,704.00 550,404.00 860,206.00
amphur_code   13,635.00 4,526.63   2,139.24   1,026.00   3,009.00   3,807.00   5,504.00   8,602.00  
province_code 13,635.00 45.18      21.45      10.00      30.00      38.00      55.00      86.00     
latitude      13,635.00 15.50      2.35       10.58      14.53      16.49      18.14      18.24     
longitude     13,635.00 101.73     2.18       98.57      100.57     102.05     104.35     104.50    
humid         13,635.00 80.43      12.44      40.90      71.00      82.40      90.60      100.00    
light         13,635.00 33.20      34.51      0.00       0.00       16.00      68.00      96.00     
pm10          6,841.00  7.66       8.45       1.00       3.00       6.00       10.00      368.00    
pm2.5         6,841.00  6.45       8.42       0.00       2.00       4.00       9.00       3

## Correlation

In [ ]:
corr = df.corr()
corr_unstacked = corr.unstack()
corr_unstacked_sorted = corr_unstacked.sort_values(kind="quicksort", ascending=False)
corr_df=pd.Series.to_frame(corr_unstacked_sorted, name='correlation')  #Convert Series to DataFrame and name correlation column accordingly. 
high_corr_features =  corr_df[corr_df.correlation != 1]  #Remove all 1s that correspond to self correlation
high_corr_features.unstack()['correlation'][['temp']].sort_values(by='temp')

,temp
humid,-0.90
amphur_code,-0.19
tambon_code,-0.19
province_code,-0.19
pm2.5,-0.13
pm10,-0.13
rainfall,-0.07
wind_speed,0.01
latitude,0.07
wind_direct,0.08


In [ ]:
df.shape

(13635, 19)

## Drop outliers

In [ ]:
df_clean = df.copy()

# drop outliers
df_clean = df_clean[df_clean['wind_speed'] < 15000]

# drop na
df_clean = df_clean.dropna(subset=['temp'], axis=0, how='any')

In [ ]:
df_clean.shape

(13633, 19)

## Feature Engineering
- hour: เวลาในแต่ละวัน เช่น 8โมง 9โมง บ่ายโมง บ่ายสอง 
- month: เดือนที่เท่าไหร่
- day: วันที่เท่าไหร่ใน 1 เดือน ตอนหลังมาคิดอีกทีควรใช้ วันในรอบปี น่าจะได้ผลดีขึ้น
- one hot encoding: mac, station_name, tambon_namt, amphur_namt, province_namt, month, hour, day
- drop คอลัมน์ที่ไม่ได้ใช้ หรือคอลัมน์ที่แปลงเป็น one hot แล้ว หรือคอลัมน์ที่ มีทิศทางเดียวกัน เช่น station id กับ station name

In [ ]:
def prepData(df_clean):
    df_prep = df_clean.drop(['pm10', 'pm2.5'], axis=1)
    df_prep['time'] = pd.to_datetime(df_prep.time)
    df_prep['hour'] = df_prep['time'].dt.hour
    df_prep['month'] = df_prep['time'].dt.month
    df_prep['day'] = df_prep['time'].dt.day

    df_prep = (
        df_prep
        .join
        (pd.get_dummies(df_prep['mac']
        .explode(), drop_first=True)
        .groupby(level=0)
        .sum()
        )
        .join
        (pd.get_dummies(df_prep['station_name']
        .explode(), drop_first=True)
        .groupby(level=0)
        .sum()
        )
        .join
        (pd.get_dummies(df_prep['tambon_namt']
        .explode(), drop_first=True)
        .groupby(level=0)
        .sum()
        )
        .join
        (pd.get_dummies(df_prep['amphur_namt']
        .explode(), drop_first=True)
        .groupby(level=0)
        .sum()
        .add_prefix('amphur_')
        )
        .join
        (pd.get_dummies(df_prep['province_namt']
        .explode(), drop_first=True)
        .groupby(level=0)
        .sum()
        )
        .join
        (pd.get_dummies(df_prep['month']
        .explode(), drop_first=True)
        .groupby(level=0)
        .sum()
        .add_prefix('month_')
        )
        .join
        (pd.get_dummies(df_prep['hour']
        .explode(), drop_first=True)
        .groupby(level=0)
        .sum()
        .add_prefix('hour_')
        )
        .join
        (pd.get_dummies(df_prep['day']
        .explode(), drop_first=True)
        .groupby(level=0)
        .sum()
        .add_prefix('day_')
        )
        .drop(['hour', 
               'month',
               'day',
               'mac', 
               'station_name', 
               'tambon_namt', 
               'tambon_code', 
               'amphur_namt', 
               'amphur_code', 
               'province_namt', 
               'province_code', 
               'time'], axis=1)
    )
    return df_prep

In [ ]:
df_train_prep = prepData(df_clean)
df_train_prep

,latitude,longitude,humid,light,rainfall,wind_direct,wind_speed,temp,3C71BF15DB04,3C71BF164F90,3C71BF17CDBC,3C71BF18CEA4,3C71BF18EA64,3C71BF1B1E28,807D3AF57920,บ้านนาสะแบง_2,บ้านสำโรงเกียรติ_2,โรงเรียนท่าข้ามวิทยา,โรงเรียนบ้านนา,โรงเรียนสรรพวิทยาคม,โรงเรียนสามเสนนอก(ประชาราษฎร์อนุกูล),โรงเรียนหนองสูงสามัคคีวิทยา_2,ดินแดง,ท่าข้าม,นาสะแบง,บักดอง,สถาน,หนองสูงเหนือ,แม่สอด,amphur_ดินแดง,amphur_ท่าแซะ,amphur_นาน้อย,amphur_ศรีวิไล,amphur_หนองสูง,amphur_แม่สอด,amphur_โนนไทย,ชุมพร,ตาก,นครราชสีมา,น่าน,บึงกาฬ,มุกดาหาร,ศรีสะเกษ,month_6,month_7,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,day_2,day_3,day_4,day_5,day_6,day_7,day_8,day_9,day_10,day_11,day_12,day_13,day_14,day_15,day_16,day_17,day_18,day_19,day_20,day_21,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30,day_31
0,15.11,102.05,70.60,59.00,0.00,45.00,4.30,32.60,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,15.11,102.05,62.30,76.00,0.00,45.00,1.90,37.90,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,15.11,102.05,90.30,0.00,0.00,135.00,3.20,26.40,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,15.11,102.05,91.70,0.00,0.00,157.50,0.00,25.60,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,15.11,102.05,59.10,71.00,0.00,180.00,2.30,36.20,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13630,16.49,104.35,97.90,0.00,4.20,0.00,0.00,23.50,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13631,16.49,104.35,80.60,0.00,0.00,135.00,2.90,19.30,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13632,16.49,104.35,87.90,37.00,0.00,45.00,3.00,24.50,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
13633,16.49,104.35,89.90,0.00,0.00,315.00,0.00,27.30,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_test = pd.read_csv('/content/IOT_Test.csv', index_col='id')
df_test_prep = prepData(df_test)
df_test_prep

,latitude,longitude,humid,light,rainfall,wind_direct,wind_speed,3C71BF15DB04,3C71BF164F90,3C71BF17CDBC,3C71BF18CEA4,3C71BF18EA64,3C71BF1B1E28,807D3AF57920,บ้านนาสะแบง_2,บ้านสำโรงเกียรติ_2,โรงเรียนท่าข้ามวิทยา,โรงเรียนบ้านนา,โรงเรียนสรรพวิทยาคม,โรงเรียนสามเสนนอก(ประชาราษฎร์อนุกูล),โรงเรียนหนองสูงสามัคคีวิทยา_2,ดินแดง,ท่าข้าม,นาสะแบง,บักดอง,สถาน,หนองสูงเหนือ,แม่สอด,amphur_ดินแดง,amphur_ท่าแซะ,amphur_นาน้อย,amphur_ศรีวิไล,amphur_หนองสูง,amphur_แม่สอด,amphur_โนนไทย,ชุมพร,ตาก,นครราชสีมา,น่าน,บึงกาฬ,มุกดาหาร,ศรีสะเกษ,month_6,month_7,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,day_2,day_3,day_4,day_5,day_6,day_7,day_8,day_9,day_10,day_11,day_12,day_13,day_14,day_15,day_16,day_17,day_18,day_19,day_20,day_21,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30,day_31
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,15.11,102.05,88.90,1.00,0.00,180.00,2.30,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,15.11,102.05,85.00,40.00,0.30,135.00,1.50,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,15.11,102.05,88.90,7.00,0.00,157.50,0.00,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,15.11,102.05,81.90,66.00,0.00,135.00,0.20,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,15.11,102.05,64.90,71.00,0.00,225.00,2.40,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3409,16.49,104.35,87.00,0.00,0.00,315.00,0.00,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3410,16.49,104.35,71.70,81.00,0.00,180.00,3.10,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3411,16.49,104.35,86.50,2.00,0.00,0.00,0.60,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


# Split Data

In [ ]:
#Split into features and target
from sklearn.model_selection import train_test_split
X = df_train_prep.drop('temp', axis = 1)
y = df_train_prep['temp']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 20)

# Build Deep learning model

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential


print(tf.__version__)

2.9.2


In [ ]:
df_train_prep.describe().transpose()[['mean', 'std']]

,mean,std
latitude,15.50,2.35
longitude,101.73,2.18
humid,80.43,12.44
light,33.20,34.51
rainfall,0.14,1.11
wind_direct,161.85,100.35
wind_speed,3.43,127.69
temp,29.08,4.34
3C71BF15DB04,0.12,0.33
3C71BF164F90,0.12,0.32


## Normalization

In [ ]:
normalizer = tf.keras.layers.Normalization(axis=-1)

In [ ]:
normalizer.adapt(np.array(X_train))

## Basic DNN (use MAE as loss function)

In [ ]:
def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(97, activation='relu'),
      layers.Dense(128, activation='relu'),
      layers.Dense(128, activation='relu'),
      layers.Dense(512, activation='relu'),
      layers.Dropout(0.1),
      layers.Dense(1, activation='linear')
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.0002),
                metrics=['mae'])
  return model

In [ ]:
dnn_model = build_and_compile_model(normalizer)

In [ ]:
dnn_model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 97)               195       
 n)                                                              
                                                                 
 dense_67 (Dense)            (None, 97)                9506      
                                                                 
 dense_68 (Dense)            (None, 128)               12544     
                                                                 
 dense_69 (Dense)            (None, 128)               16512     
                                                                 
 dense_70 (Dense)            (None, 512)               66048     
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                     

## Stop loss

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

## Train model

In [ ]:
%%time
history = dnn_model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    verbose=1, epochs=300,
    callbacks=[callback])

Epoch 1/300
273/273 [==============================] - 2s 6ms/step - loss: 28.2130 - mae: 28.2130 - val_loss: 27.2583 - val_mae: 27.2583
Epoch 2/300
273/273 [==============================] - 1s 5ms/step - loss: 24.7472 - mae: 24.7472 - val_loss: 21.2163 - val_mae: 21.2163
Epoch 3/300
273/273 [==============================] - 1s 5ms/step - loss: 13.4550 - mae: 13.4550 - val_loss: 5.6574 - val_mae: 5.6574
Epoch 4/300
273/273 [==============================] - 1s 5ms/step - loss: 4.6308 - mae: 4.6308 - val_loss: 4.0947 - val_mae: 4.0947
Epoch 5/300
273/273 [==============================] - 1s 5ms/step - loss: 3.8113 - mae: 3.8113 - val_loss: 3.4871 - val_mae: 3.4871
Epoch 6/300
273/273 [==============================] - 1s 5ms/step - loss: 3.3431 - mae: 3.3431 - val_loss: 3.1115 - val_mae: 3.1115
Epoch 7/300
273/273 [==============================] - 1s 5ms/step - loss: 3.0357 - mae: 3.0357 - val_loss: 2.8622 - val_mae: 2.8622
Epoch 8/300
273/273 [==============================] - 1s 5

## Val pred

In [ ]:
dnn_model.evaluate(X_test, y_test, verbose=1)

86/86 [==============================] - 0s 2ms/step - loss: 0.7499 - mae: 0.7499


[0.7499344944953918, 0.7499344944953918]

## Test pred

In [ ]:
dnn_model.predict(df_test_prep).flatten()

107/107 [==============================] - 0s 2ms/step


array([25.311941, 28.135862, 25.913277, ..., 26.649694, 24.902853,
       36.304676], dtype=float32)

# Submission

In [ ]:
results = pd.DataFrame(dnn_model.predict(df_test_prep).flatten())

107/107 [==============================] - 0s 2ms/step


In [ ]:
df_submit = pd.read_csv('/content/IOT_Submit.csv')

In [ ]:
submission = pd.concat([df_submit.loc[:2],
          df_submit.loc[3:,['id']].join(results).rename(columns={0:'temp'})], axis=0).set_index('id')

In [ ]:
submission

,temp
id,
1,25.60
2,30.10
3,24.70
4,28.49
5,34.87
...,...
3409,25.14
3410,32.71
3411,26.65


In [ ]:
#submission.to_csv('submission.csv')

In [ ]:
#!kaggle competitions submit -c ss3-hackathon-online-iots-and-robotics -f submission.csv -m "hello kaggle"

100% 77.1k/77.1k [00:01<00:00, 67.9kB/s]
Successfully submitted to Hackathon Online: IoTs and Robotics